<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/parser/parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recipe1M parser

In [1]:
!pip install quantulum3
!pip install stemming
!pip install pattern

     |████████████████████████████████| 10.7 MB 13.3 MB/s 
     |████████████████████████████████| 101 kB 11.0 MB/s 
  Created wheel for stemming: filename=stemming-1.0.1-py3-none-any.whl size=11138 sha256=870b4a4ea92ad2e63366ae1ba7327ce57a2bbe96d7bfec48d73ceb7c478d5e97
  Stored in directory: /root/.cache/pip/wheels/6b/e5/e2/c52ebc0a5b53fd82b00cc385e57bb1c90bd50e5f54ddbc06d1
Successfully built stemming
     |████████████████████████████████| 22.2 MB 122.5 MB/s 
     |████████████████████████████████| 87 kB 5.7 MB/s 
     |████████████████████████████████| 81 kB 8.2 MB/s 
     |████████████████████████████████| 5.6 MB 65.2 MB/s 
     |████████████████████████████████| 5.6 MB 80.9 MB/s 
     |████████████████████████████████| 419 kB 56.8 MB/s 
     |████████████████████████████████| 104 kB 88.0 MB/s 
     |████████████████████████████████| 3.6 MB 59.2 MB/s 
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=038a92eb9efded0ae56cb704cfa17f46a7b621d47290

In [3]:
# Add GDrive
from google.colab import drive
import sys
drive.mount('/content/drive/')
sys.path.append('/content/drive/My Drive/Datasets/Recipe1M/')

Mounted at /content/drive/


In [1]:
import pandas as pd
import re
from recipe import Recipe

#FILE_DIR = '../'
FILE_DIR = '/content/drive/My Drive/Datasets/Recipe1M/'

## Recipe1M data
Recipe1M comes with various json files containing crawled recipes from the web. For our project, two of them are interesting:
* layer1.json: Contains all recipes to their full extend
  
  ![layer1](https://github.com/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/dataset-analysis/layer1_puml.png?raw=1)

* det_ingrs.json: Only contains recipe ID, parsed ingredients and validity flag for parsing 
  
  ![det_ingrs](https://github.com/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/dataset-analysis/det_ingrs_puml.png?raw=1)

In our first attempt we want to make use of the parsed ingredient list and only consider recipes, where all ingredients are marked valid. The parsed ingredients don't contain amounts, so our parser has to kind of merge content of both files. Extracting ingredients from one and their amount and unit from the other file.

## Preprocessing
Removal of all invalid sets from ingredient and full data json to reduce memory. Use pickle instead of json.

In [2]:
ingredient_data = None
recipe_data = None
ingredient_file = FILE_DIR + 'det_ingrs.json'
layer1_file = FILE_DIR + 'layer1.json'

ingredient_out = FILE_DIR + '2022_01_30/det_ingrs_valid.pkl'
layer1_out = FILE_DIR + '2022_01_30/layer1_valid.pkl'

# Get data, set id as used index and drop unnecessary information
ingredient_data = pd.read_json(ingredient_file).set_index('id')
recipe_data = pd.read_json(layer1_file).drop(columns=['url', 'partition']).set_index('id')

# Drop recipes with more than 20 ingredients
indices = ingredient_data[[True if len(row) > 20 else False for row in ingredient_data['valid']]].index
ingredient_data = ingredient_data.drop(indices)
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with more than 20 ingredients')

# Drop recipes with more than 30 instructions
indices = recipe_data[[True if len(row) > 30 else False for row in recipe_data['instructions']]].index
ingredient_data = ingredient_data.drop(indices)
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with more than 30 instructions')

# Removal of all elements in ingredient json which contain invalid entries according to the data set
# Get indices of ingredients which contain false valid flags 
indices = ingredient_data[[True if any(x == False for x in row) else False for row in ingredient_data['valid']]].index
ingredient_data = ingredient_data.drop(indices).drop(columns=['valid'])
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with invalid ingredients')

indices = []
# Replace fractions in raw ingredients
fractionRegex = re.compile('[0-9]+/[0-9]+')
# Replace patterns like "1 12" which is intended to be 1 1/2 (1.5)
doubleRegex = re.compile('[1-9] [1-9][2-9]')

# Bring both datasets together
recipe_data['ingredients_parsed'] = ingredient_data['ingredients']

for idx, recipe in recipe_data.iterrows():
    # Put every information into one data set
    for ingredient in recipe['ingredients']:
        text = ingredient['text']
        if text:
            fractions = re.findall(fractionRegex, text)
            doubleNumbers = re.findall(doubleRegex, text)
            if fractions:
                numbers = fractions[0].split('/')
                float_representation = int(numbers[0])/int(numbers[1])
                text = text.replace(fractions[0], str(float_representation))
                ingredient['text'] = text
            elif doubleNumbers:
                numbers = doubleNumbers[0].split(' ')
                float_representation = int(numbers[0]) + int(numbers[1][0])/int(numbers[1][1])
                text = text.replace(doubleNumbers[0], str(float_representation))
                ingredient['text'] = text
        elif idx not in indices:
            # To be removed later
            indices.append(idx)



# Remove empty ingredients from data
for index in indices:
    recipe_data.loc[index]['ingredients'] = [elem for elem in recipe_data.loc[index]['ingredients'] if elem['text']]
    recipe_data.loc[index]['ingredients_parsed'] = [elem for elem in recipe_data.loc[index]['ingredients_parsed'] if elem['text']]

# Save data to pickle (it's faster)
recipe_data.to_pickle(layer1_out)

recipe_data.head(10)



Removed 18626 recipes with more than 20 ingredients
Removed 14270 recipes with more than 30 instructions
Removed 149732 recipes with invalid ingredients


,ingredients,title,instructions,ingredients_parsed
id,,,,
000033e39b,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",Dilly Macaroni Salad Recipe,[{'text': 'Cook macaroni according to package ...,"[{'text': 'elbow macaroni'}, {'text': 'America..."
000035f7ed,"[{'text': '8 tomatoes, quartered'}, {'text': '...",Gazpacho,[{'text': 'Add the tomatoes to a food processo...,"[{'text': 'tomatoes'}, {'text': 'kosher salt'}..."
00003a70b1,"[{'text': '2.5 cups milk'}, {'text': '1.5 cups...",Crunchy Onion Potato Bake,[{'text': 'Preheat oven to 350 degrees Fahrenh...,"[{'text': 'milk'}, {'text': 'water'}, {'text':..."
00004320bb,[{'text': '1 (3 ounce) package watermelon gela...,Cool 'n Easy Creamy Watermelon Pie,"[{'text': 'Dissolve Jello in boiling water.'},...","[{'text': 'watermelon gelatin'}, {'text': 'boi..."
0000631d90,"[{'text': '12 cup shredded coconut'}, {'text':...",Easy Tropical Beef Skillet,"[{'text': 'In a large skillet, toast the cocon...","[{'text': 'shredded coconut'}, {'text': 'lean ..."
000075604a,"[{'text': '2 Chicken thighs'}, {'text': '2 tsp...",Kombu Tea Grilled Chicken Thigh,[{'text': 'Pierce the skin of the chicken with...,"[{'text': 'chicken thighs'}, {'text': 'tea'}, ..."
00007bfd16,"[{'text': '6 -8 cups fresh rhubarb, or'}, {'te...",Strawberry Rhubarb Dump Cake,[{'text': 'Put ingredients in a buttered 9 x 1...,"[{'text': 'fresh rhubarb'}, {'text': 'frozen r..."
000095fc1d,"[{'text': '8 ounces, weight Light Fat Free Van...",Yogurt Parfaits,[{'text': 'Layer all ingredients in a serving ...,"[{'text': 'non - fat vanilla yogurt'}, {'text'..."
0000973574,"[{'text': '2 cups flour'}, {'text': '1 tablesp...",Zucchini Nut Bread,"[{'text': 'Sift dry ingredients.'}, {'text': '...","[{'text': 'flour'}, {'text': 'cinnamon'}, {'te..."


## Plausibilty check (postprocessing)
Steps in postprocessing are:
1. Check recipe on ingredient units that obviously don't fit to the context of recipes.
2. Check for implausible amounts (In the data set, we see high amounts such as "14" which in the original recipe are meant to be a fraction (1/4) but have been wrongly crawled. Convert these amounts to their intended amount.
3. Convert units to metrical system and unify (e.g. all volumes to ml, all masses to g). Only leave cup and spoons as they are.

Return None if recipe is implausible, else return the processed recipe

In [2]:

allowed_units = ['cup', 'tablespoon', 'teaspoon', '', 'pound-mass', 'ounce', 'millilitre', 'gram' ,'package',
 'pinch', 'quart', 'drop' ,'pint', 'inch', 'litre', 'fluid ounce', 'kilogram', 'centimetre',
 'gallon', 'centilitre', 'decilitre', 'millimetre', 'dessertspoon', 'milligram']

allowed_amounts = {'cup': 80, 'tablespoon': 50, 'teaspoon': 50, '': 50, 'pound-mass': 80, 'ounce': 50, 'millilitre': 5000, 'gram': 5000 ,'package': 50,
 'pinch': 20, 'quart': 20, 'drop': 20 ,'pint': 20, 'inch': 50, 'litre': 10, 'fluid ounce': 100, 'kilogram': 10, 'centimetre': 300,
 'gallon': 10, 'centilitre': 1000, 'decilitre': 100, 'millimetre': 3000, 'dessertspoon': 50, 'milligram': 5000}

# Units where fractions are most likely
fraction_units = ['cup', 'tablespoon', 'teaspoon', 'dessertspoon', 'pound-mass']



def postprocess(recipe):
    for ingredient in recipe.ingredients:
        # Some of the resulting units don't fit well to usual recipe units
        if not ingredient['unit'] in allowed_units:
            print(f'Recipe {recipe.id} is not added bc {ingredient} is filtered!')
            return None
        
        # If amount is too high, we don't consider the recipe
        if ingredient['amount'] > allowed_amounts[ingredient['unit']]:
            print(f'Recipe {recipe.id} is not added bc {ingredient} exceeds limits!')
            return None

        # Try to reconstruct fractions
        if ingredient['amount'] > 11 and ingredient['unit'] in fraction_units:
            denominator = ingredient['amount'] % 10
            if denominator != 0:
                numerator = ingredient['amount'] // 10
                ingredient['amount'] = numerator / denominator

        # Convert units
        if ingredient['unit'] == 'litre':
            ingredient['unit'] = 'millilitre'
            ingredient['amount'] = ingredient['amount'] * 1000
        elif ingredient['unit'] == 'kilogram':
            ingredient['unit'] = 'gram'
            ingredient['amount'] = ingredient['amount'] * 1000
        elif ingredient['unit'] == 'centilitre':
            ingredient['unit'] = 'millilitre'
            ingredient['amount'] = ingredient['amount'] * 10
        elif ingredient['unit'] == 'decilitre':
            ingredient['unit'] = 'millilitre'
            ingredient['amount'] = ingredient['amount'] * 100
        elif ingredient['unit'] == 'millimetre':
            ingredient['unit'] = 'centimetre'
            ingredient['amount'] = ingredient['amount'] / 10
        elif ingredient['unit'] == 'dessertspoon':
            ingredient['unit'] = 'teaspoon'
            ingredient['amount'] = ingredient['amount'] * 2
        elif ingredient['unit'] == 'milligram':
            ingredient['unit'] = 'gram'
            ingredient['amount'] = ingredient['amount'] / 1000

    return recipe

## Actual parsing

In [3]:
recipes = []
# Sort by ID, 
recipe_data = pd.read_pickle(FILE_DIR + '2022_01_30/layer1_valid.pkl')
print(f'Total number of recipes: {len(recipe_data)}')
recipe_data.head(5)


Total number of recipes: 847092


,ingredients,title,instructions,ingredients_parsed
id,,,,
000033e39b,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",Dilly Macaroni Salad Recipe,[{'text': 'Cook macaroni according to package ...,"[{'text': 'elbow macaroni'}, {'text': 'America..."
000035f7ed,"[{'text': '8 tomatoes, quartered'}, {'text': '...",Gazpacho,[{'text': 'Add the tomatoes to a food processo...,"[{'text': 'tomatoes'}, {'text': 'kosher salt'}..."
00003a70b1,"[{'text': '2.5 cups milk'}, {'text': '1.5 cups...",Crunchy Onion Potato Bake,[{'text': 'Preheat oven to 350 degrees Fahrenh...,"[{'text': 'milk'}, {'text': 'water'}, {'text':..."
00004320bb,[{'text': '1 (3 ounce) package watermelon gela...,Cool 'n Easy Creamy Watermelon Pie,"[{'text': 'Dissolve Jello in boiling water.'},...","[{'text': 'watermelon gelatin'}, {'text': 'boi..."
0000631d90,"[{'text': '12 cup shredded coconut'}, {'text':...",Easy Tropical Beef Skillet,"[{'text': 'In a large skillet, toast the cocon...","[{'text': 'shredded coconut'}, {'text': 'lean ..."


In [4]:
j = 0
fileIndex = 0
for idx, raw_recipe in recipe_data.iterrows():
    
    recipe = Recipe(idx, raw_recipe['title'])
    recipe.parse_ingredients(raw_recipe['ingredients_parsed'])
    recipe.parse_instructions(raw_recipe['instructions'])
    recipe.get_ingredient_amounts(raw_recipe['ingredients'])

    # Append postprocessed recipe
    if (postprocess(recipe)):
        recipes.append(recipe)

    j += 1
    if j % 10000 == 0:
      print(f'Progress: {j}')

    # Save every 100000 recipes
    if len(recipes) == 100000:
        # Create data frame in the end (according to Stackoverflow this is faster)                
        df = pd.DataFrame([r.to_dict() for r in recipes]).set_index('id')
        path = FILE_DIR + f'2022_01_30/recipes_valid_{fileIndex}'
        fileIndex += 1
        df.to_pickle(path + '.pkl')
        del df
        recipes = []

# Save remaining data
if len(recipes) > 0:
    df = pd.DataFrame([r.to_dict() for r in recipes]).set_index('id')
    path = FILE_DIR + f'2022_01_30/recipes_valid_{fileIndex}'
    df.to_pickle(path + '.pkl')
    df.head(20)
    del df


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your ow

Streaming output truncated to the last 5000 lines.
Recipe 3bccc33e9a is not added bc {'amount': 113.0, 'unit': 'cup', 'ingredient': 'pasta sauce'} exceeds limits!
Recipe 3bcdb2bc6d is not added bc {'amount': 12.0, 'unit': 'litre', 'ingredient': 'oil'} exceeds limits!
Recipe 3be2900aa4 is not added bc {'amount': 812.0, 'unit': '', 'ingredient': 'prosciutto'} exceeds limits!
Recipe 3bf781be24 is not added bc {'amount': 395.0, 'unit': '', 'ingredient': 'white chocolate'} exceeds limits!
Recipe 3c078042c4 is not added bc {'amount': 3000.0, 'unit': '', 'ingredient': 'kraft processed cheese slice'} exceeds limits!
Recipe 3c0952fb7e is not added bc {'amount': 64.0, 'unit': '', 'ingredient': 'wheat cracker'} exceeds limits!
Recipe 3c0b75b1d6 is not added bc {'amount': 100.0, 'unit': '', 'ingredient': 'apple juice'} exceeds limits!
Recipe 3c0c731a6a is not added bc {'amount': 12.0, 'unit': 'gallon', 'ingredient': 'whole milk'} exceeds limits!
Recipe 3c16502a4c is not added bc {'amount': 171.0, 